<a name="0"></a>
# CSE 4705: Assignment 03 - Local Search - 8-queens - Part 1

In this lab you will:
- Implement various local search algorithms that we've discussed in class, including:

1.  Steepest ascent hill climbing
2.  Stochastic hill climbing
3.  First choice hill climbing
4.  Random restart hill climbing.

# Outline
- [ 0 - Assignment ](#0)
- [_0.1 - Problem Statement](#0.1)
- [_0.2 - Objective Function - Number of Attacking Pairs](#0.2)
- [_0.3 - Queens Assignment Representation](#0.3)
- [ 1 - Problem 1 - Steepest Ascent Hill Climbing ](#1)
- [_1.1 - Exercise - Function Implementation - Count attacking pairs for a given queens assignment](#1.1)
- [_1.2 - Exercise - Function Implementation - Count attacking pairs for successors of queens assignment](#1.2)
- [_1.3 - Exercise - Function Implementation - Steepest Ascent Hill Climb](#1.3)
- [ 2 - Problem 2 - Stochastic Hill Climbing ](#2)
- [_2.1 - Exercise - Function Implementation - Probability distribution based on queens assignment](#2.1)
- [_2.2 - Exercise - Function Implementation - Stochastic Hill Climbing](#2.2)
- [ 3 - Problem 3 - First Choice Hill Climbing ](#3)
- [_3.1 - Exercise - Function Implementation - First choice](#3.1)
- [_3.2 - Exercise - Function Implementation - First choice hill climbing](#3.2)
- [ 4 - Problem 4 - Random Restart Hill Climbing ](#4)
- [_4.1 - Exercise - Function Implementation - Random Restart Hill Climbing](#4.1)
- [5 - Congratulations!](#5)


<a name="0.1"></a>
##  0.1 Problem Statement

In this assignment you will implement a number of local search algorithms for solving the n-queens problem.  As we discussed in class, this problem consists of determining how to arrange queen pieces on an n x n chess board so that no two queens are attacking each other.  The diagram below shows one of a number of successful arrangmeents of queens on an 8 x 8 chess board:

<img src="images/eight_queens.png" width="400" height="400">

Your algorithms will start with a random arrangement of queens on the board, and then will utilize the algorithms to approach, or hopefully, successfully find, a goal state in which no two queens are attacking each other.

The algorithms you will implement include the following:

1.  [25 points] Hill Climbing (Steepest Ascent)
2.  [30 points] Stochastic Hill Climbing
3.  [30 points] First Choice Hill Climbing
4.  [15 points] Random Restart Hill Climbing

<a name="0.2"></a>
## 0.2 Objective Function - Number of Attacking Pairs

As discussed in class, a commonly used metric for the objective function for the 8-queens problem is the number of pairs of attacking queens for a given 8-queens assignment (i.e., 8-queens arrangement on the board).  The goal state is a queens assignment for which attack pairs = 0.

In the slide deck from Lecture 04 - Local Search, there is a calculation for the attack pairs metric for the given queens assignment for each square on the board, as shown below.  We'll discuss this example in more detail a bit later.

<img src="images/attacking_pairs.png" width="700" height="700">

You will use the attack pairs metric for guiding the search in your algorithm implementations.

<a name="0.3"></a>
## 0.3 Queens Assignment Representation

You will use a numpy array to represent the assignment of a set of locations for queens on the chess board.  It will take on a form consistent with the following example:

$$ \text{queens} = \text{ndarray([3, 2, 1, 4, 3, 2, 1, 2])}  $$

This numpy array indicates the row position for each queen located in each of the 8 columns on the board.  Note that the row indices are 0-based.  So, the bottom row is indicated with a 0, the second row from the bottom by a 1, and so on up through top row indicated by a 7.

The example above gives the representation for the arrangement of queens in the image above.  That is, queens[0] = 3 indicates the queen in the first column is in the fourth row.  Meanwhile, queens[1] = 2 indicates the queen in the second column is in the third row.  This same reasoning follows for the rest of the 8 queens in the array.


In [1]:
import pandas as pd
import numpy as np
from random import choices
from queue import PriorityQueue

<a name="1"></a>
## 1 - Steepest Ascent Hill Climbing

[25 points]

You will implement the steepest ascent hill climbing algorithm in the cells below.

<a name="1.1"></a>
### 1.1 Exercise - Function Implementation - Count attacking pairs for a given queens assignment

[10 points]

Below, the attack_paris() function is intended to return the number of pairs of queens attacking each other for a given queens assignment, passed in as an input argument, in the form of an numpy array (as described in section 0.3.  

Implement this function according to the specs given for the function.  

There are some simple (but not comprehensive) unit tests after the function to indicate whether you are on the right track.

In [4]:
def attack_pairs(queens):
    """
    computes number of pairs of attacking queens
    Args:
      queens (ndarray (n, ))  : represents the assignment of queens on the board.  n = 8 for the 8-queens problem.
    Returns
      attack_pairs scaler     : number of pairs of attacking queens for the given queens assignment on the board.
    """
    attack_pairs = 0


    ### START CODE HERE
    for i in range(len(queens)):
      for j in range(i+1,len(queens)):
        if queens[i] == queens[j] or abs(queens[i] - queens[j]) == abs(i - j):
          attack_pairs += 1
    ### END CODE HERE

    return attack_pairs

In [5]:
# UNIT TEST 1 - attack_pairs()

queens = np.array([3, 2, 1, 4, 3, 2, 1, 2])
ap = attack_pairs(queens)

print(f'number of attacking pairs for [3, 2, 1, 4, 3, 2, 1, 2]: {ap}')

number of attacking pairs for [3, 2, 1, 4, 3, 2, 1, 2]: 17


**Expected Reult:** number of attacking pairs for [3, 2, 1, 4, 3, 2, 1, 2]: 17

In [6]:
# UNIT TEST 2 - attack_pairs()

queens = np.array([0, 2, 1, 4, 3, 2, 1, 2])
ap = attack_pairs(queens)

print(f'number of attacking pairs for [0, 2, 1, 4, 3, 2, 1, 2]: {ap}')

number of attacking pairs for [0, 2, 1, 4, 3, 2, 1, 2]: 14


**Expected Reult:** number of attacking pairs for [0, 2, 1, 4, 3, 2, 1, 2]: 14

<a name="1.2"></a>
### 1.2 Exercise - Function Implementation - Count attacking pairs for successors of queens assignment

[5 points]

The attack_paris_board() function computes the number of pairs of queens attacking each other when you've moved one queen in one column to a different row within the same column.  (Moves of queens within their same columns are what constitutes successors for the purposes of our approach.)

Refer to our example:

<img src="images/attacking_pairs.png" width="700" height="700">

This diagram shows, for example, that if we move the queen in the first column from the 3rd row (from the bottom) to the 4th row, the number of attacking pairs will change from its current value of 17 to 15.  On the other hand, as an another example, if we move the queen in the third column from its current position in the 2nd row to the top row, the number of pairs of attacking queen changes from 17 to 12.

So, attack_pairs_board() computes all of these numbers and returns them in the form of an n x n Numpy array.  

Implement the function in the space provided.  **You should make use of the attack_pairs() function you implemented, above, in your code for this function.**

The unit test below will help you determine whether your implementation is on the right track.


In [153]:
def attack_pairs_board(queens):
    """
    computes the number of pairs of attacking queens for each successor queen assignment to the one passed in as an
    input argument.
    Args:
      queens (ndarray (n, ))   : represents the assignment of queens on the board.  n = 8 for the 8-queens problem.
    Returns
      counts (ndarray (n, n))  : number of pairs of attacking queens for when the queen of each column is moved from
                                 its current row to the row of each respective cell.
    """

    n = len(queens)

    ### START CODE HERE

    # counts to be returned in the form of an n x n Numpy array.
    counts = np.zeros((n, n))
    for col in range(n):
      for row in range(n):
        iteration = queens.copy()
        iteration[col] = row
        counts[row][col] = attack_pairs(iteration)
    ### END CODE HERE

    return counts

In [154]:
# UNIT TEST 1 - attack_pairs_board()

queens = np.array([3, 2, 1, 4, 3, 2, 1, 2])
ap_board = attack_pairs_board(queens)

print(f'successors attacking pairs for [3, 2, 1, 4, 3, 2, 1, 2]: \n\n{ap_board}')

successors attacking pairs for [3, 2, 1, 4, 3, 2, 1, 2]: 

[[14. 14. 13. 17. 12. 14. 12. 18.]
 [18. 14. 17. 15. 15. 14. 17. 16.]
 [17. 17. 16. 18. 15. 17. 15. 17.]
 [17. 14. 17. 15. 17. 14. 16. 16.]
 [15. 14. 14. 17. 13. 16. 13. 16.]
 [14. 12. 18. 13. 15. 12. 14. 14.]
 [14. 16. 13. 15. 12. 14. 12. 16.]
 [18. 12. 14. 13. 13. 12. 14. 14.]]


**Expected Reult:**

successors attacking pairs for [3, 2, 1, 4, 3, 2, 1, 2]:

[[14. 14. 13. 17. 12. 14. 12. 18.] <br>
 [18. 14. 17. 15. 15. 14. 17. 16.] <br>
 [17. 17. 16. 18. 15. 17. 15. 17.] <br>
 [17. 14. 17. 15. 17. 14. 16. 16.] <br>
 [15. 14. 14. 17. 13. 16. 13. 16.] <br>
 [14. 12. 18. 13. 15. 12. 14. 14.] <br>
 [14. 16. 13. 15. 12. 14. 12. 16.] <br>
 [18. 12. 14. 13. 13. 12. 14. 14.]]

In [155]:
# UNIT TEST 2 - attack_pairs_board()

queens = np.array([0, 2, 1, 4, 3, 2, 1, 2])
ap_board = attack_pairs_board(queens)

print(f'successors attacking pairs for [0, 2, 1, 4, 3, 2, 1, 2]: \n\n{ap_board}')

successors attacking pairs for [0, 2, 1, 4, 3, 2, 1, 2]: 

[[14. 13. 12. 14. 11. 12. 10. 16.]
 [18. 13. 14. 12. 13. 11. 14. 13.]
 [17. 14. 15. 15. 13. 14. 12. 14.]
 [17. 11. 14. 12. 14. 10. 12. 12.]
 [15. 11. 12. 14. 12. 13. 10. 13.]
 [14. 10. 15. 10. 13. 10. 11. 11.]
 [14. 14. 11. 11. 10. 11. 10. 13.]
 [18. 10. 12. 10. 10.  9. 11. 12.]]


**Expected Reult:**

successors attacking pairs for [0, 2, 1, 4, 3, 2, 1, 2]:

[[14. 13. 12. 14. 11. 12. 10. 16.] <br>
 [18. 13. 14. 12. 13. 11. 14. 13.] <br>
 [17. 14. 15. 15. 13. 14. 12. 14.] <br>
 [17. 11. 14. 12. 14. 10. 12. 12.] <br>
 [15. 11. 12. 14. 12. 13. 10. 13.] <br>
 [14. 10. 15. 10. 13. 10. 11. 11.] <br>
 [14. 14. 11. 11. 10. 11. 10. 13.] <br>
 [18. 10. 12. 10. 10.  9. 11. 12.]]

<a name="1.3"></a>
### 1.3 Exercise - Function Implementation - Steepest Ascent Hill Climb

[10 points]

The steepest_ascent_hill_climb() function implements the algorithm after which it was named, where at each state it moves to an adjacent state offering a minimum value of attacking pairs among the set of successors.  

**You should make use of the attack_pairs_board() function above in your logic for choosing a successor state and for determining whether you've reached a local minimum.**

In [203]:
def steepest_ascent_hill_climb(n):
    """
    performs a steepest ascent hill climb toward a goal state of a queens assignment (represented in the form of a
    Numpy array of size (n, )) in which there are no pairs of queens attacking each other.  Not every execution
    of this function will result in a success - often a local optimum will be reached (i.e., a local min in which
    the number of attacking pairs is > 0, but no neighbors offer any improvement.

    Args:
      n (scalar))                     : dimension of the board.  For 8-queens, n = 8 (but we could use this to
                                        solve say, 10-queens)
    Returns
      current_attack_pairs (scalar)   : count of attacking pairs of the local optimum it found (0 if goal state found)
      queens (ndarray (n, ))          : locally optimum queens assignment, or, if attack pairs = 0, a globally optimum
                                        assignment
    """

    # start with a random assignment of queens on the board.
    queens = np.random.randint(n, size=n)


    ### START CODE HERE
    while True:
      current_value = attack_pairs(queens)
      neighbor_grid = attack_pairs_board(queens)

      nr, nc = np.unravel_index(np.argmin(neighbor_grid), neighbor_grid.shape)
      neighbor_value = neighbor_grid[nr][nc]
      if neighbor_value >= current_value:
        return current_value, queens
      queens[nc] = nr

    ### END CODE HERE



In [204]:
# UNIT TEST 1 - steepest_ascent_hill_climb()

# This test runs steepest ascent 100 times, giving us the chance to to observe the frequency with which
# it arrives at a solution for 100 randomly chosen starting queen assignments.  We know from the literature
# that the overall average is about 14%.

np.random.seed(0)  # reset seed to produce the same set of starting queen assignments with every execution

num_successes = 0
for i in range(100):
    attack_pairs_count, queens = steepest_ascent_hill_climb(8)
    if attack_pairs_count == 0:
        print(f'Success: {queens}')
        num_successes += 1

print(f'\nNumber of successes: {num_successes}')

Success: [6 0 2 7 5 3 1 4]
Success: [2 6 1 7 5 3 0 4]
Success: [4 1 3 6 2 7 5 0]
Success: [4 0 7 5 2 6 1 3]
Success: [3 7 4 2 0 6 1 5]
Success: [3 5 0 4 1 7 2 6]
Success: [1 3 5 7 2 0 6 4]
Success: [2 5 3 1 7 4 6 0]
Success: [6 4 2 0 5 7 1 3]
Success: [6 2 7 1 4 0 5 3]
Success: [3 5 7 1 6 0 2 4]
Success: [3 1 4 7 5 0 2 6]

Number of successes: 12


**Expected Result:**

Success: [6 0 2 7 5 3 1 4] <br>
Success: [2 6 1 7 5 3 0 4] <br>
Success: [4 1 3 6 2 7 5 0] <br>
Success: [4 0 7 5 2 6 1 3] <br>
Success: [3 7 4 2 0 6 1 5] <br>
Success: [3 5 0 4 1 7 2 6] <br>
Success: [1 3 5 7 2 0 6 4] <br>
Success: [2 5 3 1 7 4 6 0] <br>
Success: [6 4 2 0 5 7 1 3] <br>
Success: [6 2 7 1 4 0 5 3] <br>
Success: [3 5 7 1 6 0 2 4] <br>
Success: [3 1 4 7 5 0 2 6] <br> <br>
Number of successes: 12

<a name="2"></a>
## 2 - Stochastic Hill Climbing

[30 points]

You will implement the stochastic hill climbing algorithm in the cells below.

<a name="2.1"></a>
### 2.1 Exercise - Function Implementation - Probability distribution based on queens assignment

[10 points]

Stochastic hill climbing involves selecting successors from a probability distribution instead of picking the one that has the largest improvement in the objective function. Therefore, in order to implement this technique, we need to build a function that returns a probability distribution upon which our selection of a state's successor will be based.

The probability distribution will be developed using the following approach:

1.  Determine the maximum number of attacking pairs possible for a set of n queens on a board.  From you days in
CSE 2500 you may recall that this is n "choose" 2, that is:

$$\text{worst case attack pairs count} = {n \choose 2} = \frac{n (n-1)}{2}$$

2.  Determine the fitness for each successor cell on the board according to the following formula:

$$ \text{successors_fitness} = \text{worst case attack pairs count} - \text{successors_counts}$$

You should use the attack_pairs_board() function you developed above to find the array of successors counts values for all the cells on the board, for a given queens assignment.  

This formula will be applied to every cell on the board, to each respective successor count.  For example, for an 8-queens instance, you should have an 8 x 8 array of successor count values (from calling attack_pairs_board()) to which you should broadcast the fitness calculation above to get an 8 x 8 grid of successor_fitness values.

3.  Scale the successor_fitness array with an constant, $k$, that prescribes the ratio of the max fitness value over the min fitness value, that is:

$$k = \text{max(successors_fitness)} / \text{min(successors_fitness)}$$

This value of $k$ will be pre-determined and will serve as an input to this function for scaling the probabilities in your distribution to be developed by this function.

4.  Calculate the scaled successor fitness values as follows:

$$ \text{scaled_successors_fitness} = \frac{\text{successors_fitness} \cdot (k-1)}{(x_2 - x_1)} + \frac{x_2 - k \cdot x_1}{(x_2 - x_1)} $$

where

$$x_2 = \text{max(successors_fitness)}$$
and
$$x_1 = \text{min(successors_fitness)}$$

This step should yield an n x n ndarray where the following principle holds:

$$ \text{max(successors_fitness)} = k \times \text{min(successors_fitness)}$$

5.  Build the probabilities by dividing these scaled successor fitness values by their sum.  

$$ \text{probabilities} = \frac{\text{scaled_successors_fitness}}{\text{scaled_successors_fitness.sum()}} $$

This yields an n x n ndarray of values between 0 and 1 which serves as the distribution returned by the function.

Notice that cells with lower attack pair values will be assigned higher probabilities and vice versa and that the sum of these values is 1 (as required for a probability distribution).

In [221]:
def successors_probs(queens, k):
    """
    returns a probability distribution whose values correspond to the attack pair counts for a queens arrangement
    that is passed in as an input argument.  That is, cells with lower attack pair counts are assigned higher
    probabilities and those with higher counts are assigned lower probabilities.

    Args:
      queens ((n, ) ndarray)         : queens assignment on a board
      k (scalar)                     : scaling factor for probabilities.  (max_prob = k x min_prob - see above)

    Returns
      probs ((n**2, ) ndarray)         : 1D array of probs whose length is n**2, giving a probability for each cell
                                       in the n x n grid of successors.
    """
    n = len(queens)

    ### BEGIN CODE HERE

    worst_case_attack_pairs_count = n * (n - 1)/2

    successor_board = attack_pairs_board(queens)
    successor_fitness = np.array(worst_case_attack_pairs_count - successor_board)

    x2 = np.max(successor_fitness)
    x1 = np.min(successor_fitness)

    scaled_successors_fitness = (successor_fitness * (k - 1) + (x2 - k * x1)) / (x2 - x1)

    probs = scaled_successors_fitness / np.sum(scaled_successors_fitness)

    ### END CODE HERE

    return probs

In [222]:
# UNIT TEST 1 - successors_probs()

queens = np.array([3, 2, 1, 4, 3, 2, 1, 2])
successors_probs(queens, k=20)

array([[0.01942207, 0.01942207, 0.02392231, 0.00592136, 0.02842255,
        0.01942207, 0.02842255, 0.00142113],
       [0.00142113, 0.01942207, 0.00592136, 0.01492184, 0.01492184,
        0.01942207, 0.00592136, 0.0104216 ],
       [0.00592136, 0.00592136, 0.0104216 , 0.00142113, 0.01492184,
        0.00592136, 0.01492184, 0.00592136],
       [0.00592136, 0.01942207, 0.00592136, 0.01492184, 0.00592136,
        0.01942207, 0.0104216 , 0.0104216 ],
       [0.01492184, 0.01942207, 0.01942207, 0.00592136, 0.02392231,
        0.0104216 , 0.02392231, 0.0104216 ],
       [0.01942207, 0.02842255, 0.00142113, 0.02392231, 0.01492184,
        0.02842255, 0.01942207, 0.01942207],
       [0.01942207, 0.0104216 , 0.02392231, 0.01492184, 0.02842255,
        0.01942207, 0.02842255, 0.0104216 ],
       [0.00142113, 0.02842255, 0.01942207, 0.02392231, 0.02392231,
        0.02842255, 0.01942207, 0.01942207]])

**Expected Result:**

array([0.01942207, 0.01942207, 0.02392231, 0.00592136, 0.02842255, <br>
       0.01942207, 0.02842255, 0.00142113, 0.00142113, 0.01942207, <br>
       0.00592136, 0.01492184, 0.01492184, 0.01942207, 0.00592136, <br>
       0.0104216 , 0.00592136, 0.00592136, 0.0104216 , 0.00142113, <br>
       0.01492184, 0.00592136, 0.01492184, 0.00592136, 0.00592136, <br>
       0.01942207, 0.00592136, 0.01492184, 0.00592136, 0.01942207, <br>
       0.0104216 , 0.0104216 , 0.01492184, 0.01942207, 0.01942207, <br>
       0.00592136, 0.02392231, 0.0104216 , 0.02392231, 0.0104216 , <br>
       0.01942207, 0.02842255, 0.00142113, 0.02392231, 0.01492184, <br>
       0.02842255, 0.01942207, 0.01942207, 0.01942207, 0.0104216 , <br>
       0.02392231, 0.01492184, 0.02842255, 0.01942207, 0.02842255, <br>
       0.0104216 , 0.00142113, 0.02842255, 0.01942207, 0.02392231, <br>
       0.02392231, 0.02842255, 0.01942207, 0.01942207])

<a name="2.2"></a>
### 2.2 Exercise - Function Implementation - Stochastic Hill Climbing

[20 points]

Using the successors_probs() function you created above, you will apply it in the implementation of the stochastic hill climbing algorithm you'll code in the cell below.

In stochastic hill climbing, the algorithm picks a successor state based on a probability distribution, not on a steepest ascent metric.  You'll call the successors_probs() function to create a distribution that biases in favor of states that offer larger improvement in the attack_pairs metric, but allows the possibility of a successor with a smaller improvement, or even a negative change.  

How to terminate this algorithm?  Allow this function to iterate 1000 times.  If it finds a goal state before that, it should return the goal state queens assignment.  Otherwise, return whatever it has after 1000 iterations.  

Notice that we do not stop the algorithm if we hit a local minimum - we simply keep on picking states randomly until we hit a goal or 1000 iterations.



In [268]:
def stochastic_hill_climb(n, k):
    """
    implements the stochastic hill climbing algorithm, starting with a random queens assignment and repeatedly
    picking successor states randomly (according to a probability distribution proportionate to states' fitness
    levels) until either a goal state is found (no attacking pairs) or until 1000 iterations have been executed.

    Args:
      n  (scalar)                    : size of the board.  In 8-queens, n = 8
      k (scalar)                     : scaling factor for probabilities

    Returns
      attack_pairs (scalar)          : count of attacking pairs of queens when the algorithm is finished (0 if
                                       it finds a goal state)
      queens ((n, ) ndarray)         : queens assignment when the algorithm is finished
    """


    # start with a random assignment of queens on the board.
    queens = np.random.randint(n, size=n)

    ### START CODE HERE

    # iterate 1000 times
    for i in range(1000):
      current_value = attack_pairs(queens)
      # If it finds a goal state before that, it should return the goal state queens assignment
      if current_value == 0:
        return current_value, queens

      # Calculate our probabilities
      probs = successors_probs(queens, k)

      chosen_flattened_index = np.random.default_rng().choice(np.arange(len(probs.flatten())), p=probs.flatten())

      # retrieve 2d index in board shape via unravel
      # nr and nc are the row and column of the next chosen move
      nr, nc = np.unravel_index(chosen_flattened_index, probs.shape)
      queens[nc] = nr


    ### END CODE HERE

    return attack_pairs(queens), queens


In [269]:
# UNIT TEST 1 - stochastic_hill_climb()

# np.random.seed(0)  # reset seed to produce the same set of starting queen assignments with every execution

num_successes = 0
for i in range(100):
    attack_pairs_count, queens = stochastic_hill_climb(n = 8, k = 5000)
    if attack_pairs_count == 0:
        print(f'Success: {queens}')
        num_successes += 1

print(f'\nNumber of successes: {num_successes}')

Success: [5 1 6 0 3 7 4 2]
Success: [3 7 4 2 0 6 1 5]
Success: [5 2 4 6 0 3 1 7]
Success: [5 0 4 1 7 2 6 3]
Success: [4 2 0 5 7 1 3 6]
Success: [4 6 0 3 1 7 5 2]
Success: [3 0 4 7 1 6 2 5]
Success: [1 3 5 7 2 0 6 4]
Success: [3 1 4 7 5 0 2 6]
Success: [3 5 7 1 6 0 2 4]
Success: [4 0 7 3 1 6 2 5]
Success: [5 3 6 0 7 1 4 2]
Success: [4 6 0 3 1 7 5 2]
Success: [5 2 6 1 7 4 0 3]

Number of successes: 14


**Example Result: (your result will likely vary from this):**

Success: [5 3 6 0 2 4 1 7] <br>
Success: [5 7 1 3 0 6 4 2] <br>
Success: [3 0 4 7 1 6 2 5] <br>
Success: [7 3 0 2 5 1 6 4] <br>
Success: [2 5 1 6 4 0 7 3] <br>
Success: [0 6 4 7 1 3 5 2] <br>
Success: [2 5 1 6 0 3 7 4] <br>
Success: [4 2 0 6 1 7 5 3] <br>
Success: [2 0 6 4 7 1 3 5] <br>
Success: [2 5 3 1 7 4 6 0] <br>
Success: [3 6 4 2 0 5 7 1] <br>
Success: [5 2 4 7 0 3 1 6] <br> <br>
Number of successes: 12

<a name="3"></a>
### Problem 3 - First Choice Hill Climbing

[30 points]

You will implement the first choice hill climbing algorithm in the cells below.

<a name="3.1"></a>
### 3.1 Exercise - Function Implementation - First choice

[10 points]

Implement the first_choice() function which repeatedly picks successor states until one is found that is better than
the current state; that is, has a lower attacking pairs count than that of the current state.

This function takes a queens assignment as an input parameter and a scaling factor, k, which gives determines the
character of the probability distribution.  

**Use the attack_pairs() function, the attack_pairs_board() function, and the successors_probs() functions you implemented above to help you code the implementation for this function.**  The value of k passed in as an input
parameter is the parm you'll pass to the successors_probs function.

#### first_choice() implementation - Q:  Do we have an infinite loop concern? - A:  No, if instructions are followed...
Note that you should **not** need to be concerned with this function, first_choice(), entering an infinite loop because of a possible edge case of queens (the input array) being a local min whose attack_pairs() count is less than all successors.  This is because your first_choice_hill_climb() function (to be implemented next) should only call this
function if queens is **not** a local min.


In [315]:
def first_choice(queens, k):
    """
    returns an index value of a successor state picked randomly, but which offers an improvement in the attack pairs
    metric over the current state of the queens assignment passed in as input.  You will use the random.choices()
    function to pick a value based on the probability distribution.
    Args:
      queens ((n, ) ndarray)        : queens assignment on a board
      k  (scalar)                   : scaling factor for the probability distribution.  needed by successors_probs()
    Returns
      select_index (scalar)         : index value of the cell in the attack_pairs_board(queens) array which gives the
                                    successor chosen to the queens array passed in as input.  Note you will need
                                    to use the following to map back to a row and index in the 2D attack_pairs_board()
                                    array:  row = select_index // n,  column = select_index % n
    """


    n = len(queens)


    ### START CODE HERE



    # start at worst possible state
    current_state = attack_pairs(queens)
    rng = np.random.default_rng()


    successor_board = attack_pairs_board(queens)

    probs = successors_probs(queens, k)
    probs_flattened_indexes = np.arange(len(probs.flatten()))

    select_index = rng.choice(probs_flattened_indexes, p=probs.flatten())
    while True:
      # Choose our random index (used generator because numpy wiki dissuades using random.choice() in new code)
      chosen_flattened_index = rng.choice(probs_flattened_indexes, p=probs.flatten())

      if successor_board.flatten()[chosen_flattened_index] < current_state:
        select_index = chosen_flattened_index
        break

    ### END CODE HERE

    return select_index

In [316]:
# UNIT TEST 1 - first_choice()

# np.random.seed(0)  # reset seed to produce the same set of starting queen assignments with every execution

queens = np.array([3, 2, 1, 4, 3, 2, 1, 2])

n = len(queens)
print(f'queens: {queens}')

ap = attack_pairs(queens)
print(f'attack_pairs(queens): {ap}')

select_index = first_choice(queens, 5)
row_move = select_index // n
col_move = select_index % n

print(f'select_index: {select_index}')
print(f'row_move: {row_move}')
print(f'col_move: {col_move}')

# move to successor state (move queen...)
queens[col_move] = row_move
print(f'queens: {queens}')

ap_new = attack_pairs(queens)
print(f'attack_pairs(queens): {ap_new}')

queens: [3 2 1 4 3 2 1 2]
attack_pairs(queens): 17
select_index: 18
row_move: 2
col_move: 2
queens: [3 2 2 4 3 2 1 2]
attack_pairs(queens): 16


**Example Result: (your result will likely vary from this):**

queens: [3 2 1 4 3 2 1 2] <br>
attack_pairs(queens): 17 <br>
select_index: 5 <br>
row_move: 0 <br>
col_move: 5 <br>
queens: [3 2 1 4 3 0 1 2] <br>
attack_pairs(queens): 14

<a name="3.2"></a>
### 3.2 Exercise - Function Implementation - First choice hill climbing

[20 points]

Implement the first choice hill climbing algorihm in the cell below, utilizing the first_choice() function you coded above for choosing the successor at each step.  

Note that your implementation should *test whether the current state is a local min before calling first_choice()*.  This prevents first_choice() from entering an infinite loop, as discussed in the comments to the last exercise.

Execute the algo loop 1000 times in your implementation.  If a goal state is found, return the attack pairs count of 0 and the queens assignment.  If no goal state is found, return the attack pairs count and queens assignment at the last step of the algorithm.

In [329]:
def first_choice_hill_climb(n, k):
    """
    implements the first choice hill climbing algorithm, starting with a random queens assignment and repeatedly
    picking successor states using the first choice function until either a goal state is found (no attacking pairs)
    or until 1000 iterations have been executed.

    Args:
      n  (scalar)                    : size of the board.  In 8-queens, n = 8
      k  (scalar)                    : scaling factor for the probability distribution.  needed by successors_probs()
    Returns
      attack_pairs (scalar)          : count of attacking pairs of queens when the algorithm is finished (0 if
                                       it finds a goal state)
      queens ((n, ) ndarray)         : queens assignment when the algorithm is finished
    """

    # start with a random assignment of queens on the board.
    queens = np.random.randint(n, size=n)

    ### START CODE HERE
    for i in range(1000):
      current_value = attack_pairs(queens)
      if current_value == 0:
        return current_value, queens

      successor_board = attack_pairs_board(queens)
      current_min = np.min(successor_board.flatten())

      # if current_value != 0, and is the lowest currently in the successor board, then we have reached a local minimum
      if current_value == current_min:
        break
      else:
        flat_index = first_choice(queens, k)
        row_move = flat_index // n
        col_move = flat_index % n
        queens[col_move] = row_move

    ### END CODE HERE

    return attack_pairs(queens), queens



In [330]:
# UNIT TEST 1 - first_choice_hill_climb()

# np.random.seed(0)  # reset seed to produce the same set of starting queen assignments with every execution

num_successes = 0
for i in range(100):
    attack_pairs_count, queens = first_choice_hill_climb(n = 8, k = 10)
    if attack_pairs_count == 0:
        print(f'Success: {queens}')
        num_successes += 1
print(f'\nNumber of successes: {num_successes}')

Success: [4 2 0 6 1 7 5 3]
Success: [1 6 2 5 7 4 0 3]
Success: [6 3 1 7 5 0 2 4]
Success: [4 2 0 6 1 7 5 3]
Success: [3 6 2 7 1 4 0 5]
Success: [5 1 6 0 2 4 7 3]
Success: [1 4 6 0 2 7 5 3]
Success: [6 2 7 1 4 0 5 3]
Success: [3 6 4 2 0 5 7 1]
Success: [2 5 7 1 3 0 6 4]
Success: [3 5 7 2 0 6 4 1]
Success: [3 6 4 2 0 5 7 1]
Success: [6 3 1 7 5 0 2 4]
Success: [4 0 7 5 2 6 1 3]
Success: [3 1 6 2 5 7 4 0]
Success: [1 6 4 7 0 3 5 2]
Success: [4 6 1 5 2 0 7 3]
Success: [2 6 1 7 4 0 3 5]

Number of successes: 18


**Example Result: (your result will likely vary from this):**

Success: [1 7 5 0 2 4 6 3] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [1 7 5 0 2 4 6 3] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [4 7 3 0 6 1 5 2] <br>
Success: [4 7 3 0 6 1 5 2] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [4 7 3 0 2 5 1 6] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [4 7 3 0 6 1 5 2] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [4 1 5 0 6 3 7 2] <br>
Success: [4 1 5 0 6 3 7 2] <br> <br>

Number of successes: 15

<a name="4"></a>
### Problem 4 - Random Restart Hill Climbing

[15 points]

You will implement the random restart hill climbing algorithm in the cells below.

<a name="4.1"></a>
### 4.1 Exercise - Function Implementation - Random Restart Hill Climbing

[15 points]

Random restart hill climbing is essentially repeated executions of the steepest ascent hill climbing algorithm.  Implement this algorithm in the cell below.

In [331]:
def random_restart_hill_climb(n, attempts):
    """
    implements the random restart hill climbing algorithm, executing the steepest ascent hill climbing algorithm
    until a goal state is found or until an attempt limit has been reached.

    Args:
      n  (scalar)                    : size of the board.  In 8-queens, n = 8
      attempts  (scalar)             : the number of attempts to take at the steepest ascent hill climbing algorithm
    Returns
      attack_pairs (scalar)          : count of attacking pairs of queens when the algorithm is finished (0 if
                                       it finds a goal state)
      queens ((n, ) ndarray)         : queens assignment when the algorithm is finished, goal state if one if found
    """

    best_ap = 9999
    best_queens = np.zeros(8)

    ### START CODE HERE
    for i in range(attempts):
      attack_pairs, queens = steepest_ascent_hill_climb(n)
      if attack_pairs == 0:
        return attack_pairs, queens
      elif attack_pairs < best_ap:
        best_ap = attack_pairs
        best_queens = queens

    ### END CODE HERE

    return best_ap, best_queens


In [332]:
# UNIT TEST 1 - random_restart_hill_climb()

np.random.seed(0)  # reset seed to produce the same set of starting queen assignments with every execution

num_successes = 0
for i in range(100):
    attack_pairs_count, queens = random_restart_hill_climb(n = 8, attempts=7)
    if attack_pairs_count == 0:
        print(f'Success: {queens}')
        num_successes += 1
print(f'\nNumber of successes: {num_successes}')

Success: [6 0 2 7 5 3 1 4]
Success: [2 6 1 7 5 3 0 4]
Success: [4 1 3 6 2 7 5 0]
Success: [4 0 7 5 2 6 1 3]
Success: [3 7 4 2 0 6 1 5]
Success: [3 5 0 4 1 7 2 6]
Success: [1 3 5 7 2 0 6 4]
Success: [2 5 3 1 7 4 6 0]
Success: [6 4 2 0 5 7 1 3]
Success: [6 2 7 1 4 0 5 3]
Success: [3 5 7 1 6 0 2 4]
Success: [3 1 4 7 5 0 2 6]
Success: [3 1 7 4 6 0 2 5]
Success: [1 7 5 0 2 4 6 3]
Success: [0 5 7 2 6 3 1 4]
Success: [4 6 0 2 7 5 3 1]
Success: [4 7 3 0 2 5 1 6]
Success: [0 5 7 2 6 3 1 4]
Success: [3 1 6 2 5 7 4 0]
Success: [1 4 6 0 2 7 5 3]
Success: [1 6 4 7 0 3 5 2]
Success: [5 0 4 1 7 2 6 3]
Success: [2 5 7 1 3 0 6 4]
Success: [4 2 0 6 1 7 5 3]
Success: [1 6 2 5 7 4 0 3]
Success: [1 5 7 2 0 3 6 4]
Success: [5 2 6 1 3 7 0 4]
Success: [4 7 3 0 6 1 5 2]
Success: [2 6 1 7 4 0 3 5]
Success: [5 2 4 7 0 3 1 6]
Success: [3 1 7 4 6 0 2 5]
Success: [4 7 3 0 2 5 1 6]
Success: [3 1 7 5 0 2 4 6]
Success: [4 2 0 5 7 1 3 6]
Success: [3 7 0 4 6 1 5 2]
Success: [6 2 0 5 7 4 1 3]
Success: [3 6 4 2 0 5 7 1]
S

**Expected Result:**

Success: [6 0 2 7 5 3 1 4] <br>
Success: [2 6 1 7 5 3 0 4] <br>
Success: [4 1 3 6 2 7 5 0] <br>
Success: [4 0 7 5 2 6 1 3] <br>
Success: [3 7 4 2 0 6 1 5] <br>
Success: [3 5 0 4 1 7 2 6] <br>
Success: [1 3 5 7 2 0 6 4] <br>
Success: [2 5 3 1 7 4 6 0] <br>
Success: [6 4 2 0 5 7 1 3] <br>
Success: [6 2 7 1 4 0 5 3] <br>
Success: [3 5 7 1 6 0 2 4] <br>
Success: [3 1 4 7 5 0 2 6] <br>
Success: [3 1 7 4 6 0 2 5] <br>
Success: [1 7 5 0 2 4 6 3] <br>
Success: [0 5 7 2 6 3 1 4] <br>
Success: [4 6 0 2 7 5 3 1] <br>
Success: [4 7 3 0 2 5 1 6] <br>
Success: [0 5 7 2 6 3 1 4] <br>
Success: [3 1 6 2 5 7 4 0] <br>
Success: [1 4 6 0 2 7 5 3] <br>
Success: [1 6 4 7 0 3 5 2] <br>
Success: [5 0 4 1 7 2 6 3] <br>
Success: [2 5 7 1 3 0 6 4] <br>
Success: [4 2 0 6 1 7 5 3] <br>
Success: [1 6 2 5 7 4 0 3] <br>
Success: [1 5 7 2 0 3 6 4] <br>
Success: [5 2 6 1 3 7 0 4] <br>
Success: [4 7 3 0 6 1 5 2] <br>
Success: [2 6 1 7 4 0 3 5] <br>
Success: [5 2 4 7 0 3 1 6] <br>
Success: [3 1 7 4 6 0 2 5] <br>
Success: [4 7 3 0 2 5 1 6] <br>
Success: [3 1 7 5 0 2 4 6] <br>
Success: [4 2 0 5 7 1 3 6] <br>
Success: [3 7 0 4 6 1 5 2] <br>
Success: [6 2 0 5 7 4 1 3] <br>
Success: [3 6 4 2 0 5 7 1] <br>
Success: [1 4 6 0 2 7 5 3] <br>
Success: [4 1 7 0 3 6 2 5] <br>
Success: [2 5 1 6 0 3 7 4] <br>
Success: [5 7 1 3 0 6 4 2] <br>
Success: [4 6 0 3 1 7 5 2] <br>
Success: [0 4 7 5 2 6 1 3] <br>
Success: [4 1 3 6 2 7 5 0] <br>
Success: [0 6 4 7 1 3 5 2] <br>
Success: [2 5 7 0 4 6 1 3] <br>
Success: [3 6 0 7 4 1 5 2] <br>
Success: [5 2 6 1 3 7 0 4] <br>
Success: [6 4 2 0 5 7 1 3] <br>
Success: [1 4 6 0 2 7 5 3] <br>
Success: [5 1 6 0 2 4 7 3] <br>
Success: [5 7 1 3 0 6 4 2] <br>
Success: [4 1 3 5 7 2 0 6] <br>
Success: [5 7 1 3 0 6 4 2] <br>
Success: [5 2 4 7 0 3 1 6] <br>
Success: [5 1 6 0 2 4 7 3] <br>
Success: [5 1 6 0 2 4 7 3] <br>
Success: [1 7 5 0 2 4 6 3] <br> <br>

Number of successes: 58

<a name="5"></a>
## 5 Congratulations!
In this lab you:
- implemented four significant local search, hill climbing algorithms - steepest ascent, stochastic, first choice, and randomized restart.